## Ponemos a prueba el modelo SVM entrenado con todo train (X,y) frente a solo entrenado con X_train

# Paso 1: Cargar el Modelo Entrenado


In [1]:
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import pickle  

# Cargar el modelo entrenado

with open('../modelos_y_vectorizadores/modelo_SVM_todo_train.pkl', 'rb') as archivo: 
     loaded_model_svm = pickle.load(archivo)

# Verificar que el modelo se haya cargado correctamente
print("Modelo cargado:", loaded_model_svm)


Modelo cargado: SVC(kernel='linear', random_state=42)


In [2]:
import joblib
# Cargar el vectorizador
vectorizador = joblib.load('../modelos_y_vectorizadores/vectorizador.pkl')


# Paso 2: Preprocesar test.csv

Ahora, procede a cargar y preprocesar test.csv de la misma manera que lo hicimos con train.csv

In [3]:
import pandas as pd

# Cargar test.csv
test_df = pd.read_csv('../data_es/test.csv')

test_df.head()


,mensaje,tipo
0,Tu devolucion esta siendo procesada,ham
1,Gana dinero por compartir tus opiniones,spam
2,Recordatorio de tu cita con el nutricionista,ham
3,Quema grasa abdominal con este suplemento,spam
4,Agradecemos tu preferencia,ham


In [4]:
test_df.describe()

,mensaje,tipo
count,209,209
unique,209,2
top,Tu devolucion esta siendo procesada,ham
freq,1,109


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   mensaje  209 non-null    object
 1   tipo     209 non-null    object
dtypes: object(2)
memory usage: 3.4+ KB


In [6]:
# Contar los mensajes duplicados
duplicados = test_df[test_df.duplicated(['mensaje'], keep=False)]

# Mostrar los mensajes duplicados
print("Mensajes duplicados:")
print(duplicados)


Mensajes duplicados:
Empty DataFrame
Columns: [mensaje, tipo]
Index: []


In [7]:
# Mostrar los nombres de las columnas del dataset
print(test_df.columns)

Index(['mensaje', 'tipo'], dtype='object')


In [8]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stopwords_sp = stopwords.words('spanish')

# Agregar palabras adicionales que no aporten significado en este contexto específico
stopwords_sp.extend(['este', 'nuestro', 'con', 'para', 'esta'])  # Podemos ajustar esta lista según necesitemos

# Función para limpiar el texto y filtrar stopwords
def limpiar_texto_con_stopwords(texto):
    texto = re.sub(r'\W', ' ', texto) # Eliminar caracteres no alfanuméricos
    texto = texto.lower() # Convertir a minúsculas
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios extra
    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_sp]
    return ' '.join(palabras_filtradas)

# Aplicar limpieza con filtro de stopwords a los mensajes
test_df['mensaje_limpio_stopwords'] = test_df['mensaje'].apply(limpiar_texto_con_stopwords)

# Mostrar los mensajes limpios con filtrado de stopwords
print(test_df[['mensaje', 'mensaje_limpio_stopwords']].head(10))

                                          mensaje  \
0             Tu devolucion esta siendo procesada   
1         Gana dinero por compartir tus opiniones   
2    Recordatorio de tu cita con el nutricionista   
3       Quema grasa abdominal con este suplemento   
4                      Agradecemos tu preferencia   
5             Obten tu score crediticio sin costo   
6    Firma el acuerdo de confidencialidad adjunto   
7         Aprovecha descuentos exclusivos para ti   
8             Confirmamos que recibimos tu pedido   
9  Desata tu potencial sexual con estas pastillas   

                 mensaje_limpio_stopwords  
0             devolucion siendo procesada  
1         gana dinero compartir opiniones  
2         recordatorio cita nutricionista  
3        quema grasa abdominal suplemento  
4                 agradecemos preferencia  
5            obten score crediticio costo  
6  firma acuerdo confidencialidad adjunto  
7         aprovecha descuentos exclusivos  
8            confirm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Vectorización de los Mensajes de Test
limpios de test_df usando el vectorizador cargado:

In [9]:
# Aplicar transformación usando el vectorizador cargado
X_test = vectorizador.transform(test_df['mensaje_limpio_stopwords'])


# Hacer Predicciones con el Modelo
El modelo Naive Bayes cargado para hacer predicciones sobre X_test:

In [19]:
''' #Resultado solo entrenado por X-train
# 
# from sklearn.metrics import classification_report, confusion_matrix

# Predecir con el modelo cargado
y_pred_test = loaded_model.predict(X_test)

# Convertir etiquetas numéricas a strings ('ham' y 'spam')
y_pred_str = ['spam' if pred == 1 else 'ham' for pred in y_pred_test]

# Obtener métricas de evaluación
print("Reporte de Clasificación:")
print(classification_report(test_df['tipo'], y_pred_str))

# Obtener y mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(test_df['tipo'], y_pred_str))'''


Reporte de Clasificación:
              precision    recall  f1-score   support

         ham       0.91      0.83      0.87       109
        spam       0.83      0.91      0.87       100

    accuracy                           0.87       209
   macro avg       0.87      0.87      0.87       209
weighted avg       0.87      0.87      0.87       209

Matriz de Confusión:
[[90 19]
 [ 9 91]]


# Reporte de Clasificación de NB solo entrenado por X-train:

|              | precision | recall | f1-score | support |
|--------------|-----------|--------|----------|---------|
| ham          | 0.91      | 0.83   | 0.87     | 109     |
| spam         | 0.83      | 0.91   | 0.87     | 100     |
| **accuracy** |           |        | 0.87     | 209     |
| macro avg    | 0.87      | 0.87   | 0.87     | 209     |
| weighted avg | 0.87      | 0.87   | 0.87     | 209     |

# Matriz de Confusión:

|      | ham | spam |
|------|-----|------|
| ham  | 90  | 19   |
| spam | 9   | 91   |


In [10]:
''' # Resultado de NB entrenado con todo X,y

from sklearn.metrics import classification_report, confusion_matrix

# Predecir con el modelo cargado
y_pred_test = loaded_model.predict(X_test)

# Convertir etiquetas numéricas a strings ('ham' y 'spam')
y_pred_str = ['spam' if pred == 1 else 'ham' for pred in y_pred_test]

# Obtener métricas de evaluación
print("Reporte de Clasificación: de NB")
print(classification_report(test_df['tipo'], y_pred_str))

# Obtener y mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(test_df['tipo'], y_pred_str))''' 

Reporte de Clasificación:
              precision    recall  f1-score   support

         ham       0.92      0.86      0.89       109
        spam       0.86      0.92      0.89       100

    accuracy                           0.89       209
   macro avg       0.89      0.89      0.89       209
weighted avg       0.89      0.89      0.89       209

Matriz de Confusión:
[[94 15]
 [ 8 92]]


# Reporte de Clasificación de NB entrenado con todo X,y:

|              | precision | recall | f1-score | support |
|--------------|------------|--------|----------|---------|
| ham          | 0.92       | 0.86   | 0.89     | 109     |
| spam         | 0.86       | 0.92   | 0.89     | 100     |
| **accuracy** |            |        | 0.89     | 209     |
| macro avg    | 0.89       | 0.89   | 0.89     | 209     |
| weighted avg | 0.89       | 0.89   | 0.89     | 209     |

# Matriz de Confusión:

|      | ham | spam |
|------|-----|------|
| ham  | 94  | 15   |
| spam | 8   | 92   |


In [14]:
# Resultado de SVM entrenado con todo X,y

from sklearn.metrics import classification_report, confusion_matrix

# Predecir con el modelo cargado
y_pred_test = loaded_model_svm.predict(X_test)

# Convertir etiquetas numéricas a strings ('ham' y 'spam')
y_pred_str = ['spam' if pred == 1 else 'ham' for pred in y_pred_test]

# Obtener métricas de evaluación
print("Reporte de Clasificación: de SVM")
print(classification_report(test_df['tipo'], y_pred_str))

# Obtener y mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(test_df['tipo'], y_pred_str))

Reporte de Clasificación: de SVM
              precision    recall  f1-score   support

         ham       0.91      0.85      0.88       109
        spam       0.85      0.91      0.88       100

    accuracy                           0.88       209
   macro avg       0.88      0.88      0.88       209
weighted avg       0.88      0.88      0.88       209

Matriz de Confusión:
[[93 16]
 [ 9 91]]


## Conclusiones

# Precisión y Recall:

Ambos modelos muestran un rendimiento similar con Naive Bayes teniendo una ligera ventaja en la clase "ham" en términos de precisión, recall, y F1-score.

Naive Bayes tiene una precisión ligeramente mayor en general (0.89 vs 0.88).

# Matriz de Confusión:

Naive Bayes comete menos errores de clasificación en ambas clases comparado con SVM. Tiene menos falsos negativos y falsos positivos.

# Balance entre las Clases:

Ambos modelos tienen un balance adecuado entre las clases "ham" y "spam". La diferencia en las métricas no es significativa, pero Naive Bayes parece ser ligeramente mejor.

# Elección del Modelo:

Dado que Naive Bayes tiene una precisión global ligeramente mejor y comete menos errores en la matriz de confusión, se puede considerar como la mejor opción para este problema de clasificación de spam en este conjunto de datos específico.








In [19]:
# Mostrar mensajes donde el modelo NB se equivocó
test_df['prediccion'] = y_pred_str
errores = test_df[test_df['tipo'] != test_df['prediccion']]

print("Mensajes donde el modelo se equivocó:")
print(errores[['mensaje', 'tipo', 'prediccion']])


Mensajes donde el modelo se equivocó:
                                           mensaje  tipo prediccion
14               Confirma tu registro al seminario   ham       spam
24                  Tu donacion hara la diferencia   ham       spam
32          Encuentra tu equilibrio con meditacion   ham       spam
33         Danos tu opinion sobre nuestro servicio  spam        ham
35               No calificaste para este prestamo   ham       spam
56   Protegemos tus compras con garantia extendida   ham       spam
77        Regresa a tu infancia con estos juguetes  spam        ham
93       Se parte del record Guinness con nosotros  spam        ham
94       Tu certificado de ingresos ya fue emitido   ham       spam
98              Lee como protegemos tu informacion   ham       spam
103   Lee nuestro analisis semanal de la industria   ham       spam
139           Monitorea tu envio con este tracking   ham       spam
149  Verifica tu numero de cuenta para el deposito  spam        ham
158       

In [17]:
# Mostrar mensajes donde el modelo SVM se equivocó
test_df['prediccion'] = y_pred_str
errores = test_df[test_df['tipo'] != test_df['prediccion']]

print("Mensajes donde el modelo se equivocó:")
print(errores[['mensaje', 'tipo', 'prediccion']])

Mensajes donde el modelo se equivocó:
                                             mensaje  tipo prediccion
14                 Confirma tu registro al seminario   ham       spam
24                    Tu donacion hara la diferencia   ham       spam
32            Encuentra tu equilibrio con meditacion   ham       spam
33           Danos tu opinion sobre nuestro servicio  spam        ham
35                 No calificaste para este prestamo   ham       spam
46   Conoce las actualizaciones a nuestras politicas   ham       spam
54         Reconfirma tu reserva para el tratamiento   ham       spam
56     Protegemos tus compras con garantia extendida   ham       spam
77          Regresa a tu infancia con estos juguetes  spam        ham
91                 Verifica tu correo para continuar   ham       spam
103     Lee nuestro analisis semanal de la industria   ham       spam
106        No olvides tu evaluacion medica de rutina   ham       spam
109                   Reclama aqui tu 2x1 para cenar

Conclusiones
Mensajes Comunes donde ambos Modelos se Equivocaron:

"Confirma tu registro al seminario" (ham -> spam)
"Tu donacion hara la diferencia" (ham -> spam)
"Encuentra tu equilibrio con meditacion" (ham -> spam)
"Danos tu opinion sobre nuestro servicio" (spam -> ham)
"No calificaste para este prestamo" (ham -> spam)
"Protegemos tus compras con garantia extendida" (ham -> spam)
"Regresa a tu infancia con estos juguetes" (spam -> ham)
"Lee nuestro analisis semanal de la industria" (ham -> spam)
"Monitorea tu envio con este tracking" (ham -> spam)
"Verifica tu numero de cuenta para el deposito" (spam -> ham)
"Haz tu reclamo de subsidio aqui" (ham -> spam)
"Renueva tu membresia a tiempo" (ham -> spam)
"Apoya esta causa solidaria con tu aporte" (spam -> ham)
"Registra tu telefono en un clic" (ham -> spam)
"Extiende tu suscripcion sin costo extra" (spam -> ham)
"Envia una selfie con tu documento" (spam -> ham)
"Salva una vida donando sangre hoy" (ham -> spam)
Mensajes donde solo uno de los Modelos se Equivocó:

NB tiene menos errores únicos, con mensajes como "Mejorar tus defensas con probioticos" (spam -> ham).
SVM tiene más errores únicos, incluyendo mensajes como "Conoce las actualizaciones a nuestras politicas" (ham -> spam) y "No olvides tu evaluacion medica de rutina" (ham -> spam).
Errores de Falsos Positivos (ham -> spam):

Ambos modelos cometen errores similares en los mensajes ham que predicen como spam.
Ejemplos comunes incluyen "Confirma tu registro al seminario" y "Tu donacion hara la diferencia".
Errores de Falsos Negativos (spam -> ham):

Ambos modelos cometen errores en mensajes spam que predicen como ham, aunque SVM tiene más ejemplos únicos.
Ejemplos comunes incluyen "Danos tu opinion sobre nuestro servicio" y "Regresa a tu infancia con estos juguetes".
Resumen
Ambos modelos tienen errores similares en algunos mensajes, especialmente en aquellos que son ambiguos y pueden ser interpretados de manera incorrecta.
SVM parece tener un mayor número de errores únicos, indicando que puede ser más sensible a ciertas características de los mensajes.
Naive Bayes (NB) muestra un rendimiento ligeramente mejor en términos de precisión y menor número de errores únicos.
Conclusión Final: Dado que ambos modelos tienen fortalezas y debilidades similares, Naive Bayes puede ser preferido debido a su menor número de errores únicos y su rendimiento ligeramente superior en las métricas generales.







In [12]:
# Mensaje a probar
mensaje_prueba = 'URGENTE ponte en contacto conmigo'

# Preprocesar el mensaje
mensaje_prueba_limpio = limpiar_texto_con_stopwords(mensaje_prueba)

# Vectorizar el mensaje
mensaje_prueba_vectorizado = vectorizador.transform([mensaje_prueba_limpio])

# Predecir con el modelo cargado
prediccion = loaded_model.predict(mensaje_prueba_vectorizado)

# Convertir la predicción a etiqueta
prediccion_etiqueta = 'spam' if prediccion[0] == 1 else 'ham'

print(f"Mensaje: {mensaje_prueba}")
print(f"Predicción: {prediccion_etiqueta}")

Mensaje: URGENTE ponte en contacto conmigo
Predicción: ham


In [13]:
# Mensaje a probar
mensaje_prueba = 'Sexo gratis'

# Preprocesar el mensaje
mensaje_prueba_limpio = limpiar_texto_con_stopwords(mensaje_prueba)

# Vectorizar el mensaje
mensaje_prueba_vectorizado = vectorizador.transform([mensaje_prueba_limpio])

# Predecir con el modelo cargado
prediccion = loaded_model.predict(mensaje_prueba_vectorizado)

# Convertir la predicción a etiqueta
prediccion_etiqueta = 'spam' if prediccion[0] == 1 else 'ham'

print(f"Mensaje: {mensaje_prueba}")
print(f"Predicción: {prediccion_etiqueta}")

Mensaje: Sexo gratis
Predicción: spam


In [16]:
# Mensaje a probar
mensaje_prueba = 'Recoge tu premio'

# Preprocesar el mensaje
mensaje_prueba_limpio = limpiar_texto_con_stopwords(mensaje_prueba)

# Vectorizar el mensaje
mensaje_prueba_vectorizado = vectorizador.transform([mensaje_prueba_limpio])

# Predecir con el modelo cargado
prediccion = loaded_model.predict(mensaje_prueba_vectorizado)

# Convertir la predicción a etiqueta
prediccion_etiqueta = 'spam' if prediccion[0] == 1 else 'ham'

print(f"Mensaje: {mensaje_prueba}")
print(f"Predicción: {prediccion_etiqueta}")

Mensaje: Recoge tu premio
Predicción: spam


In [17]:
# Mensaje a probar
mensaje_prueba = 'Quiero contactar contigo'

# Preprocesar el mensaje
mensaje_prueba_limpio = limpiar_texto_con_stopwords(mensaje_prueba)

# Vectorizar el mensaje
mensaje_prueba_vectorizado = vectorizador.transform([mensaje_prueba_limpio])

# Predecir con el modelo cargado
prediccion = loaded_model.predict(mensaje_prueba_vectorizado)

# Convertir la predicción a etiqueta
prediccion_etiqueta = 'spam' if prediccion[0] == 1 else 'ham'

print(f"Mensaje: {mensaje_prueba}")
print(f"Predicción: {prediccion_etiqueta}")

Mensaje: Quiero contactar contigo
Predicción: spam


In [18]:
# Mensaje a probar
mensaje_prueba = 'Todos los lunes llueve'
# Preprocesar el mensaje
mensaje_prueba_limpio = limpiar_texto_con_stopwords(mensaje_prueba)

# Vectorizar el mensaje
mensaje_prueba_vectorizado = vectorizador.transform([mensaje_prueba_limpio])

# Predecir con el modelo cargado
prediccion = loaded_model.predict(mensaje_prueba_vectorizado)

# Convertir la predicción a etiqueta
prediccion_etiqueta = 'spam' if prediccion[0] == 1 else 'ham'

print(f"Mensaje: {mensaje_prueba}")
print(f"Predicción: {prediccion_etiqueta}")

Mensaje: Todos los lunes llueve
Predicción: ham
